In [1]:
using TensorKit
import TensorKit.×
include("./iPEPS_Fermionic/iPEPS.jl")
include("./CTMRG_Fermionic/CTMRG.jl")
include("./Cal_Obs_Fermionic/Cal_Obs.jl")
# include("./models/Hubbard_Z2SU2.jl")
# include("./models/tJ_Z2.jl")
@show pkgversion(TensorKit)
using JLD2
using Strided
Strided.enable_threads()

pkgversion(TensorKit) = v"0.12.2"


4

In [ ]:
# pspace = GradedSpace{fSU₂}((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
pspace = Rep[U₁×SU₂]((-1, 0) => 1, (0, 1 // 2) => 1, (1, 0) => 1)
aspace = Rep[U₁×SU₂]((2, 0) => 2)

# aspacel = aspacer = aspacet = aspaceb = pspace
@show pspace
nothing

In [ ]:
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# pspace = Rep[ℤ₂]((1 => 1), (0 => 2))
aspacel = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspacer = Rep[ℤ₂×SU₂]((0, 0) => 2, (0, 1) => 2)
aspacer = aspacel
aspacet = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)
# aspaceb = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 3 // 2) => 2)
aspaceb = aspacet

# A = TensorMap(randn, ComplexF64, aspacel ⊗ aspacet ⊗ pspace, aspacer ⊗ aspaceb)
# Ms = [A, A, A, A]
# ipeps = iPEPS(Ms, 2, 2)
# Abar = bar(A)
# @show space(Abar)
# gate1 = swap_gate(space(Abar)[4], space(Abar)[5])
# @show space(gate1)
# gate2 = swap_gate(space(Abar)[1], space(Abar)[2])
# @show space(gate2)
# gate3 = swap_gate(space(A)[4], space(A)[5])
# @show space(gate3)
# gate4 = swap_gate(space(A)[1], space(A)[2])
# @show space(gate4)
# gate5 = swap_gate(space(A)[1], space(Abar)[2])
# @show space(gate5)
@assert pspace isa ElementarySpace
@show fieldnames(GradedSpace)

In [ ]:
ipeps, envs, para = load("/home/tcmp2/JuliaProjects/Hubbard_t1.0U8.0mu4.0_ipepsEnv_D10chi200.jld2", "ipeps", "envs", "para")
Lx = ipeps.Lx
Ly = ipeps.Ly
@show para[:χ]
ipepsbar = bar(ipeps)
nothing

In [3]:
pspace = Rep[ℤ₂](0 => 1, 1 => 2)
include("./models/tJ_Z2.jl")
OpZ = Z(pspace)

Fdag₊, F₊ = FdagF₊(pspace)
@tensor fdagf₊[p1, p3; p2, p4] := OpZ[p1, p1in] * Fdag₊[p1in, p2, a] * F₊[a, p3, p4]
@show Fdag₊
Fdag₋, F₋ = FdagF₋(pspace)
@show Fdag₋
@tensor fdagf₋[p1, p3; p2, p4] := OpZ[p1, p1in] * Fdag₋[p1in, p2, a] * F₋[a, p3, p4]

F₊, Fdag₊ = FFdag₊(pspace)
@tensor ffdag₊[p1, p3; p2, p4] := OpZ[p1, p1in] * F₊[p1in, p2, a] * Fdag₊[a, p3, p4]
F₋, Fdag₋ = FFdag₋(pspace)
@tensor ffdag₋[p1, p3; p2, p4] := OpZ[p1, p1in] * F₋[p1in, p2, a] * Fdag₋[a, p3, p4]

@show ishermitian(fdagf₊ - ffdag₊ + fdagf₋ - ffdag₋)

Fdag₊ = TensorMap(Rep[ℤ₂](0=>1, 1=>2) ← (Rep[ℤ₂](0=>1, 1=>2) ⊗ Rep[ℤ₂](1=>1))):
* Data for sector (Irrep[ℤ₂](0),) ← (Irrep[ℤ₂](1), Irrep[ℤ₂](1)):
[:, :, 1] =
 0.0  0.0
* Data for sector (Irrep[ℤ₂](1),) ← (Irrep[ℤ₂](0), Irrep[ℤ₂](1)):
[:, :, 1] =
 1.0
 0.0

Fdag₋ = TensorMap(Rep[ℤ₂](0=>1, 1=>2) ← (Rep[ℤ₂](0=>1, 1=>2) ⊗ Rep[ℤ₂](1=>1))):
* Data for sector (Irrep[ℤ₂](0),) ← (Irrep[ℤ₂](1), Irrep[ℤ₂](1)):
[:, :, 1] =
 0.0  0.0
* Data for sector (Irrep[ℤ₂](1),) ← (Irrep[ℤ₂](0), Irrep[ℤ₂](1)):
[:, :, 1] =
  0.0
 -1.0

ishermitian(((fdagf₊ - ffdag₊) + fdagf₋) - ffdag₋) = false


false

In [3]:
include("./models/Hubbard_Z2SU2.jl")
pspace = Rep[ℤ₂×SU₂]((0, 0) => 2, (1, 1 // 2) => 1)

OpZ = Z(pspace)
Opnd = nd(pspace)
Opn = n(pspace)
OpI = id(pspace)
Fdag, F = FdagF(pspace)
@tensor fdagf[p1, p3; p2, p4] := Fdag[p1in, p2, a] * F[a, p3, p4]
# @tensor fdagf[p1, p3; p2, p4] := Fdag[p1, p2, a] * F[a, p3, p4]

F, Fdag = FFdag(pspace)
@tensor ffdag[p1, p3; p2, p4] := F[p1in, p2, a] * Fdag[a, p3, p4]

@show ishermitian(fdagf - ffdag)

ishermitian(fdagf - ffdag) = false


false

In [5]:
using JLD2

para = load("D:/DMRG_projects/Nicklate/Rslt_2leg_mixDtJ/mixDtJLx64-Ly2-t3p0-tv0p0-Jv9p0_D1499/para.jld2")
@show para

para = Dict{String, Any}("para" => Dict{Symbol, Any}(:MPSDir => "/public/home/users/ucas001c/xzqu/DMRG_mixDtJ/mixDtJMPSstorage/mixDtJLx64-Ly2-t3p0-tv0p0-Jv9p0_D1499", :oldMPSDir => "/public/home/users/ucas001c/xzqu/DMRG_mixDtJ/mixDtJMPSstorage/mixDtJLx64-Ly2-t3p0-tv0p0-Jv9p0_D1000", :useexist => false, :lx => 64, :Jp => 1.0, :pbcy => false, :saveMPS => true, :RsltFldr => "/public/home/users/ucas001c/xzqu/DMRG_mixDtJ/Rslt_mixDtJ/mixDtJLx64-Ly2-t3p0-tv0p0-Jv9p0_D1499", :Jv => 9.0, :K => 5, :Geopath => "zigzag", :Dlist => [10, 10, 10, 30, 30, 60, 60, 100, 100, 100, 100, 100, 300, 300, 300, 500, 500, 500, 1000, 1000, 1499, 1499, 1499], :jobname => "mixDtJLx64-Ly2-t3p0-tv0p0-Jv9p0_D1499", :ncpu => 8, :ly => 2, :Ntot => 64, :pbcx => false, :tv => 0.0, :noise => [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], :Stot => 0, :mem => 30000, :symm => "CASA", :WDir => "/public/home/users/ucas001c/xzqu/DMRG_mixDtJ/mixDtJstorage/juli

Dict{String, Any} with 1 entry:
  "para" => Dict{Symbol, Any}(:MPSDir=>"/public/home/users/ucas001c/xzqu/DMRG_m…